<a href="https://colab.research.google.com/github/rajendranaidu495/OneClassClassification/blob/main/CMS_OCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. All Datasets: https://data.cms.gov/provider-summary-by-type-of-service/
2. CMS Part D datasets: : https://data.cms.gov/provider-summary-by-type-of-service/medicare-part-d-prescribers 
3. CMS Part B datasets: https://data.cms.gov/provider-summary-by-type-of-service/medicare-physician-other-practitioners

4. DMEPOS (Medicare Durable Medical Equipment, Devices & Supplies - by Referring Provider and Service): https://data.cms.gov/provider-summary-by-type-of-service/medicare-durable-medical-equipment-devices-supplies/medicare-durable-medical-equipment-devices-supplies-by-referring-provider-and-service

5. LEIE Databases https://oig.hhs.gov/exclusions/exclusions_list.asp

Research paper : https://journalofbigdata.springeropen.com/articles/10.1186/s40537-018-0138-3#auth-Matthew-Herland

To connect with goole drive for datasets. <br>
Gmail : rajendramannam9@gmail.com <br>
password: Mlds@2020

old_colab link: https://colab.research.google.com/drive/1cHpYMnuKV-i5RnBNSyzV63WH_F2mkfpk#scrollTo=AHxtFGpQImr9


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
!pip install pandasql
import pandasql as ps
import tensorflow as tf
#from pandasql import sqldf
#import pandasql as ps

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26784 sha256=c102acb6cbf8eda66e4945080eaacd12245af360c5de885767ce9476710a7362
  Stored in directory: /root/.cache/pip/wheels/63/e8/ec/75b1df467ecf57b6ececb32cb16f4e86697cbfe55cb0c51f07
Successfully built pandasql


In [3]:
partD_dataset_path = "/content/drive/MyDrive/CMS/partD/MUP_DPR_RY21_P04_V10_DY19_NPIBN_1.csv"
partb_dataset_path = "/content/drive/MyDrive/CMS/partb/MUP_PHY_R21_P04_V10_D19_Prov_Svc.csv"
DMEPOS_dataset_path = "/content/drive/MyDrive/CMS/DMEPOS/mup_dme_ry21_p05_v10_dy19_prvhpr_0.csv"
LEIE_dataset_path = "/content/drive/MyDrive/CMS/LEIE/UPDATED.csv"

In [4]:
#partb 
chunk_partb = pd.read_csv(partb_dataset_path, nrows=109999, encoding = "ISO-8859-1" ,chunksize=10000)
#chunk_partb = pd.read_csv(partb_dataset_path, encoding = "ISO-8859-1" ,chunksize=10000)
PartB_dataset_org= pd.concat(chunk_partb)

In [5]:
# For Part B, the aggregating process consists of grouping the data by NPI, provider type, gender and year, aggregating over HCPCS and place of service.

#Npi,Provider_type ,Nppes_provider_gender ,Line_srvc_cnt ,Bene_unique_cnt ,Bene_day_srvc_cnt ,Average_submitted_chrg_amt ,Average_medicare_payment_amt 
PartB_Required_columns = [ 'Rndrng_NPI', 'HCPCS_Desc']
#PartB_df = PartB_dataset_org[PartB_Required_columns].iloc[:100000]
PartB_df = PartB_dataset_org[PartB_Required_columns]

In [6]:
data = PartB_df

In [7]:
import pandas as pd

# group by and aggregate
grouped = PartB_df.groupby('Rndrng_NPI')['HCPCS_Desc'].agg(list).reset_index()


In [8]:
import tensorflow_hub as hub

# load Universal Sentence Encoder model
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# encode the HCPCS_Cd column
embeddings = embed(grouped['HCPCS_Desc'].apply(lambda x: ' '.join(map(str, x))))


In [9]:
embeddings

<tf.Tensor: shape=(11955, 512), dtype=float32, numpy=
array([[-0.05898675,  0.02538925, -0.0397375 , ..., -0.01572513,
        -0.01184889,  0.04134162],
       [-0.04671805,  0.05766295,  0.02756246, ..., -0.05370753,
        -0.05109846,  0.00385778],
       [-0.04660986,  0.00222035,  0.01059437, ...,  0.00504953,
        -0.04791808, -0.0342976 ],
       ...,
       [ 0.03983909, -0.04096157, -0.01192047, ..., -0.06129381,
         0.01601969, -0.05387108],
       [-0.05799888,  0.04076041,  0.02811489, ..., -0.00744882,
        -0.05774969, -0.01849042],
       [-0.05076572,  0.05310142,  0.0074351 , ..., -0.05335823,
        -0.01940938, -0.05378999]], dtype=float32)>

In [10]:
grouped["embeddings"] = embeddings.numpy().tolist()

In [11]:
grouped

,Rndrng_NPI,HCPCS_Desc,embeddings
0,1003000126,[Hospital observation care on day of discharge...,"[-0.05898675322532654, 0.025389252230525017, -..."
1,1003000134,[Pathology examination of tissue using a micro...,"[-0.0467180497944355, 0.057662952691316605, 0...."
2,1003000142,[Aspiration and/or injection of major joint or...,"[-0.04660986363887787, 0.002220351714640856, 0..."
3,1003000423,[Established patient office or other outpatien...,"[-0.03149782121181488, 0.03275752440094948, 0...."
4,1003000480,"[New patient office or other outpatient visit,...","[-0.04366494342684746, 0.04616127908229828, -0..."
...,...,...,...
11950,1013044122,[Application of electrical stimulation to 1 or...,"[0.06201868876814842, -0.0314965546131134, 0.0..."
11951,1013044189,[Cervical or vaginal cancer screening; pelvic ...,"[-0.026912016794085503, -0.0005915855290368199..."
11952,1013044270,"[Ground mileage, per statute mile, Ambulance s...","[0.03983909264206886, -0.04096157103776932, -0..."
11953,1013044411,[Removal of skin and tissue first 20 sq cm or ...,"[-0.057998884469270706, 0.04076040908694267, 0..."


In [12]:
grouped.dtypes

Rndrng_NPI     int64
HCPCS_Desc    object
embeddings    object
dtype: object

In [14]:
# LEIE
chunk_LEIE_dataset = pd.read_csv(LEIE_dataset_path, encoding = "ISO-8859-1" ,chunksize=1000000)
LEIE_dataset_org= pd.concat(chunk_LEIE_dataset)

<ipython-input-14-0688265392af>:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  LEIE_dataset_org= pd.concat(chunk_LEIE_dataset)


In [15]:
pd.set_option("display.max_columns", None)
LEIE_dataset_org.head()

,LASTNAME,FIRSTNAME,MIDNAME,BUSNAME,GENERAL,SPECIALTY,UPIN,NPI,DOB,ADDRESS,CITY,STATE,ZIP,EXCLTYPE,EXCLDATE,REINDATE,WAIVERDATE,WVRSTATE
0,NaN,NaN,,"#1 MARKETING SERVICE, INC",OTHER BUSINESS,SOBER HOME,NaN,0,NaN,239 BRIGHTON BEACH AVENUE,BROOKLYN,NY,11235,1128a1,20200319,0,0,NaN
1,NaN,NaN,,101 FIRST CARE PHARMACY INC,OTHER BUSINESS,PHARMACY,NaN,1972902351,NaN,"C/O 609 W 191ST STREET, APT D",NEW YORK,NY,10040,1128b8,20220320,0,0,NaN
2,NaN,NaN,,14 LAWRENCE AVE PHARMACY,PHARMACY,NaN,NaN,0,NaN,14 LAWRENCE AVENUE,SMITHTOWN,NY,11787,1128a1,19880830,0,0,NaN
3,NaN,NaN,,143 MEDICAL EQUIPMENT CO,DME COMPANY,DME - OXYGEN,NaN,0,NaN,701 NW 36 AVENUE,MIAMI,FL,33125,1128b7,19970620,0,0,NaN
4,NaN,NaN,,184TH STREET PHARMACY CORP,OTHER BUSINESS,PHARMACY,NaN,1922348218,NaN,69 E 184TH ST,BRONX,NY,10468,1128a1,20180419,0,0,NaN


In [16]:
LEIE_Required_columns = [ 'NPI', 'EXCLTYPE' , 'EXCLDATE' , 'REINDATE']
LEIE_df = LEIE_dataset_org[LEIE_Required_columns]

In [17]:
LEIE_df_a= LEIE_df[LEIE_df['NPI'] != 0]

In [18]:
len(LEIE_df_a)

6384

In [19]:
Final_withoutcodes_leie = LEIE_df_a[['NPI']]

In [20]:
Final_withoutcodes_leie.head()

,NPI
1,1972902351
4,1922348218
24,1942476080
28,1275600959
31,1891731758


In [21]:
Final_withoutcodes_leie['target'] = 1

<ipython-input-21-d91152302741>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Final_withoutcodes_leie['target'] = 1


In [22]:
Final_withoutcodes_leie.head()

,NPI,target
1,1972902351,1
4,1922348218,1
24,1942476080,1
28,1275600959,1
31,1891731758,1


In [23]:
from pandas.core.reshape.merge import string
# merge with target lables

combined = grouped.fillna(0)
Final_withoutcodes_leie = Final_withoutcodes_leie.fillna(0)
final_df = pd.merge(combined, Final_withoutcodes_leie, how = 'outer'
                    , left_on = 'Rndrng_NPI',
                    right_on = ['NPI'])
final_df = final_df.fillna(0)
final_df.Rndrng_NPI = final_df.Rndrng_NPI.astype(int)
final_df.NPI = final_df.NPI.astype(int)
final_df.target = final_df.target.astype(int)


In [24]:
final_df.head()

,Rndrng_NPI,HCPCS_Desc,embeddings,NPI,target
0,1003000126,[Hospital observation care on day of discharge...,"[-0.05898675322532654, 0.025389252230525017, -...",0,0
1,1003000134,[Pathology examination of tissue using a micro...,"[-0.0467180497944355, 0.057662952691316605, 0....",0,0
2,1003000142,[Aspiration and/or injection of major joint or...,"[-0.04660986363887787, 0.002220351714640856, 0...",0,0
3,1003000423,[Established patient office or other outpatien...,"[-0.03149782121181488, 0.03275752440094948, 0....",0,0
4,1003000480,"[New patient office or other outpatient visit,...","[-0.04366494342684746, 0.04616127908229828, -0...",0,0


In [27]:
final_df.loc[final_df['target'] ==1]

,Rndrng_NPI,HCPCS_Desc,embeddings,NPI,target
1080,1003066838,[Insertion of needle into vein for collection ...,"[-0.06360574066638947, -0.02276143804192543, 0...",1003066838,1
4100,1003278615,"[New patient home visit, typically 75 minutes,...","[-0.04274025931954384, -0.020655494183301926, ...",1003278615,1
6582,1003834433,"[Spinal tap for diagnosis, Neurobehavioral sta...","[-0.05732634663581848, -0.02771717496216297, -...",1003834433,1
7158,1003850603,[Established patient office or other outpatien...,"[-0.050545934587717056, 0.032872676849365234, ...",1003850603,1
9823,1003926270,[Injection of chemical for destruction of nerv...,"[0.01361254695802927, 0.02844863384962082, -0....",1003926270,1
...,...,...,...,...,...
18329,0,0,0,1053622035,1
18330,0,0,0,1124024435,1
18331,0,0,0,1194930149,1
18332,0,0,0,1578637385,1


In [ ]:
# dataset split

In [28]:
X_combined = final_df.drop(columns=['target'], axis=1)
y_combined = final_df.target

In [29]:
X_train_combined, X_test_combined, y_train_combined, y_test_combined = train_test_split(X_combined, y_combined, test_size=0.2, random_state=0)

In [ ]:
# Model

In [30]:
X_train_combined

,Rndrng_NPI,HCPCS_Desc,embeddings,NPI
3719,1003251810,[Routine electrocardiogram (EKG) using at leas...,"[-0.05459641292691231, 0.04196557030081749, -0...",0
14671,0,0,0,1316163579
11709,1013026665,[Vaccine for influenza for injection into musc...,"[-0.03650030866265297, 0.032774001359939575, -...",0
2252,1003136482,"[Therapeutic exercise to develop strength, end...","[0.05594021454453468, -0.026718612760305405, -...",0
2306,1003139932,[Nuclear medicine study of vessels of heart us...,"[-0.05631817877292633, -0.030798695981502533, ...",0
...,...,...,...,...
9225,1003904962,"[Blood test, clotting time, Catheter insertion...","[-0.055967047810554504, -0.021330317482352257,...",0
13123,0,0,0,1629074802
9845,1003927047,[Aspiration and/or injection of small joint or...,"[-0.05440864339470863, -0.01125000137835741, 0...",0
10799,1003991480,[Established patient office or other outpatien...,"[-0.024767352268099785, 0.052393849939107895, ...",0
